# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from matplotlib import pyplot as plt 

import re
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet');

In [ ]:
# load data from database - change path accordingly
engine = create_engine('sqlite:///../data/data_db/DisasterReponses.db')
df = pd.read_sql("SELECT * FROM DisasterReponses", engine)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# The messages will be the features, while the categories will be the target variables
X = df.message.values 
y = df[df.columns[4:]].to_numpy()

In [ ]:
# Check size of dataset
print("Complete feature dataset size = ", X.shape, "; complete target variables size = ", y.shape)

In [ ]:
# Check what's in X, y
X[0], y[0]

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens_raw = word_tokenize(text)
    
    # lemmatize and remove stop words
    tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens_raw if (word not in stopwords.words('english'))]

    return tokens

In [ ]:
# Test function for the first 5 messages
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

### 3. Split data in test/train


In [ ]:
# perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Check size of datasets - default split is 75% train / 25% test
print("Training feature dataset size = ", X_train.shape, "; training target variables size = ", y_train.shape)
print("Testing feature dataset size = ", X_test.shape, "; testing target variables size = ", y_test.shape)

### 4.a. Train a classifier
You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
# Instantiate transformers and classifier
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = MultiOutputClassifier(RandomForestClassifier())

# Fit and/or transform each to the data
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)
clf.fit(X_train_tfidf, y_train)

In [ ]:
# Transform test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

# Predict test labels
y_pred = clf.predict(X_test_tfidf)

After created the classifier, save it and the data in a pickle file

In [ ]:
# Create dictionary for pickle file
classifier_dict_1 = {'X_train':X_train,
                    'y_train':y_train,
                    'X_test':X_test,
                    'y_test':y_test,
                    'model':clf}

In [ ]:
# Save dictionary
filename = '../models/models_files/random_forest_1.pickle'
pickle.dump(classifier_dict_1, open(filename, 'wb'))

### 4.b. Load a classifier
In case the classifier was saved previously you can load it here

In [ ]:
# Load dictionary 
filename = '../models/models_files/random_forest_1.pickle'
classifier_dict_1 = pickle.load(open(filename, 'rb'))

In [ ]:
# Get content
X_train = classifier_dict_1['X_train']
y_train = classifier_dict_1['y_train']
X_test = classifier_dict_1['X_test']
y_test = classifier_dict_1['y_test']
clf_1 = classifier_dict_1['model']

In [ ]:
# Instantiate transformers
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()

In [ ]:
# Transform train data
X_train_counts = vect.fit_transform(X_train)
X_train_tfidf = tfidf.fit_transform(X_train_counts)

# Transform test data
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)

In [ ]:
# Predict test labels
y_pred = clf_1.predict(X_test_tfidf)

### 5. Evaluate classifier
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Example
print(classification_report(y_test[:,0], y_pred[:,0]))

In [ ]:
# Iterate
for ind_1 in range(y_pred.shape[1]):
    print('-----------------------------------------------------------------------------------------')
    print('Label = ', df.columns[ind_1 + 4])
    c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1],output_dict=True, zero_division=0)
    kk = list(c_rep.keys())
    for ind_2 in range(len(c_rep) - 3):
        print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
              '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']), 
              '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
              '; support =', c_rep[kk[ind_2]]['support'])

### 6. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 7.a. Train/save pipeline
- Split data into train and test sets
- Train pipeline
- Save pipeline with datasets

In [ ]:
# train classifier
pipeline.fit(X_train, y_train);

After created the classifier, save it and the data in a pickle file

In [ ]:
# Create dictionary for pickle file
classifier_dict_2 = {'X_train':X_train,
                    'y_train':y_train,
                    'X_test':X_test,
                    'y_test':y_test,
                    'pipeline':pipeline}

In [ ]:
# Save dictionary
filename = '../models/models_files/random_forest_pipeline_1.pickle'
pickle.dump(classifier_dict_2, open(filename, 'wb'))

### 7.b. Load saved pipeline

In case the classifier was saved previously you can load it here

In [ ]:
# Load dictionary 
filename = '../models/models_files/random_forest_pipeline_1.pickle'
classifier_dict_2 = pickle.load(open(filename, 'rb'))

In [ ]:
# Get content
X_train = classifier_dict_2['X_train']
y_train = classifier_dict_2['y_train']
X_test = classifier_dict_2['X_test']
y_test = classifier_dict_2['y_test']
pipeline = classifier_dict_2['pipeline']

### 8. Test your pipeline
 - Predict outcomes based on test data
 - Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
# display example of results 
print(classification_report(y_test[:,0], y_pred[:,0]))

In [ ]:
# Iterate
for ind_1 in range(y_pred.shape[1]):
    print('-----------------------------------------------------------------------------------------')
    print('Label = ', df.columns[ind_1 + 4])
    c_rep = classification_report(y_test[:,ind_1], y_pred[:,ind_1],output_dict=True, zero_division=0)
    kk = list(c_rep.keys())
    for ind_2 in range(len(c_rep) - 3):
        print('Value = ', kk[ind_2], ': precision = ', "{:.2f}".format(c_rep[kk[ind_2]]['precision']),
              '; recall = ', "{:.2f}".format(c_rep[kk[ind_2]]['recall']), 
              '; f1-s =', "{:.2f}".format(c_rep[kk[ind_2]]['f1-score']),
              '; support =', c_rep[kk[ind_2]]['support'])

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.